In [ ]:
import csv 
import json
import numpy as np

import led_config_utils
import mesh_config
from funky_lights import wavefront, led_config

from importlib import reload
reload(mesh_config)

all_segments = {}
for config in mesh_config.mesh_configs:
    # if config.mesh != '../controller/mesh/dome.obj':
    #     continue
    mesh = wavefront.load_obj(config.mesh)
    with open(config.actual_csv, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # skip header
        segments=[]

        for row in reader:
            uid = int(row[0])
            name = row[1]
            actual_num_leds = int(row[2])
            actual_length = float(row[3])
            reverse = (row[4] == 'TRUE')
            
            for group in mesh.groups:
                if group.name == name:
                    break

            group = next((g for g in mesh.groups if g.name == name), None)
            if not group:
                print("Couldn't find mesh group for %s" % name)
                continue

            data = led_config_utils.prepare_data(group)
            data_2d = led_config_utils.project_2d(data)
            clusters, labels = led_config_utils.cluster(
                data_2d, eps=config.cluster_eps, min_samples=3)

            nodes = led_config_utils.create_line_segments(data, clusters, labels)
            modelled_length = led_config_utils.line_segments_length(nodes)
            modelled_length = modelled_length -  config.start_offset -  config.end_offset
            if nodes == None:
                continue
            leds_distance = modelled_length / actual_num_leds
            points = led_config_utils.trace_line_segments(nodes, actual_num_leds, config.start_offset, leds_distance)
        
            if len(points) == 0:
                continue

            if reverse:
                points = np.flip(points, axis=0)

            segment = led_config.Segment(
                uid=uid, name=group.name, points=points, num_leds=points.shape[0], length=actual_length)
            print('Segment %s: length=%.1fm, num_clusters=%s, num_leds=%s' %
                  (segment.name, segment.length, len(clusters), segment.num_leds))
            segments.append(segment)
            all_segments[uid] = segment
            # led_config_utils.plot_segment(data, clusters, labels, points)

# Collapse some segments
UIDS_TRUNK = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
UIDS_TUSKS_RIGHT = [25,26,27,28,29,30,31,32,33,34,35]
UIDS_TUSKS_LEFT = [36,3,38,39,40,41,42,43,44,45,46]
for merge_list in [UIDS_TRUNK, UIDS_TUSKS_RIGHT, UIDS_TUSKS_LEFT]:
    merged_segment = all_segments[merge_list[0]]
    for uid in merge_list[1:]:
        segment = all_segments.pop(uid)
        merged_segment.merge(segment)

# Create LED config
config = led_config.LedConfig()
for segment in all_segments.values():
    config.led_segments.append(segment)
    config.total_num_segments += 1
    config.total_length += segment.length
    config.total_num_leds += segment.num_leds

with open('../config/led_config.json', 'w', encoding='utf-8') as f:
    json.dump(config.to_dict(), f, ensure_ascii=False, indent=4)

